# **Dash Basics**

This lab shows how to create a basic dash app.

We will use the wine dataset from the sklearn library.

In [ ]:
# Import required libraries

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dash import dash, dcc, html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
from sklearn.datasets import load_wine


In [ ]:
# load data function
def load_data():
    wine = load_wine()
    wine_df = pd.DataFrame(wine.data, columns=wine.feature_names)
    wine_df['WineType'] = [wine.target_names[t] for t in wine.target]
    return wine_df

wine_df = load_data()

In [ ]:
wine_df.head()

In [ ]:
ingredients = wine_df.drop(columns=['WineType']).columns

In [ ]:
# Compute the average wine dataframe
avg_wine_df = wine_df.groupby('WineType').mean().reset_index()

In [ ]:
# create function to create scatter chart

def create_scatter_chart(x_axis='alcohol', y_axis='malic_acid', color_encode=False):
    scatter_fig = px.scatter(wine_df, x=x_axis, y=y_axis, color='WineType' if color_encode else None,
                             title="{} vs {}".format(x_axis.capitalize(), y_axis.capitalize()),
                             labels=dict(x_axis="{}".format(x_axis), y_axis="{}".format(y_axis))
    )

    scatter_fig.update_layout(height=600)
    return scatter_fig 

In [ ]:
# create function to plot bar chart

def create_bar_chart(ingredients=['alcohol', 'malic_acid', 'ash']):
    bar_fig = px.bar(avg_wine_df, x='WineType', y=ingredients, 
                     title='Average Ingredients per Wine Type',
                     barmode='group',
                     labels=dict(variable='Ingredients', value='WineType'))
    bar_fig.update_layout(height=600)

    return bar_fig

In [ ]:
# Create widgets

# Dropdowns
x_axis_dropdown = dcc.Dropdown(id='x_axis', options=ingredients, value='alcohol', clearable=False, style={'display':'inline-block', 'width':'48%'})                               
y_axis_dropdown = dcc.Dropdown(id='y_axis', options=ingredients, value='malic_acid', clearable=False, style={'display':'inline-block', 'width':'48%'})
color_encode = dcc.Checklist(id='color_encode', options=['Color-Encode',])
multi_select = dcc.Dropdown(id='multi_select',options=ingredients,
                            value=['alcohol', 'malic_acid', 'ash'],multi=True, clearable=False)

In [ ]:
# Create a dash application

app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

In [ ]:
# Create layout for the app
app.layout = html.Div([
    html.H1('Wine Data Dashboard',
            style={'textAlign':'center', 'font-size':40}),

            html.Div('Explore relationship between various ingredients used in creation of three different types of wines (class_0, class_1, class_2)'),
            html.Br(),
            html.Br(),

            html.Div(children=[
                x_axis_dropdown,
                y_axis_dropdown,
                color_encode,            
                dcc.Graph(id='scatter_chart', figure=create_scatter_chart('alcohol', 'flavanoids')),
            ],
            style={'display':'inline-block', 'width': '48%'}),

            html.Div(children=[
                multi_select,
                html.Br(),
                dcc.Graph(id= 'bar_chart', figure=create_bar_chart(['alcohol', 'malic_acid', 'flavanoids']))
            ], style={'display':'inline-block', 'width': '48%'}),
            
],style={'padding':'30px'})

In [ ]:
# Create callbacks and its finctions

@app.callback(Output(component_id='scatter_chart', component_property='figure'),
              [Input(component_id='x_axis',component_property='value'),
               Input(component_id='y_axis',component_property='value'),
               Input(component_id='color_encode',component_property='value')]
              )
def update_scatter_chart(x_axis, y_axis, color_encode):
    return create_scatter_chart(x_axis, y_axis, color_encode)

@app.callback(Output(component_id='bar_chart',component_property='figure'),
          Input(component_id='multi_select',component_property='value'))
def update_bar_chart(ingredients):
    return create_bar_chart(ingredients)

In [ ]:
if __name__ == '__main__':
    app.run_server(jupyter_mode = 'tab')